# Acre per Crop Group - Mike Brady - Douglas

In [2]:
# import warnings
# warnings.filterwarnings("ignore")

import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import scipy
import scipy.signal
import os, os.path

from datetime import date
import datetime
import time

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb

import sys


# to move files from one directory to another
import shutil

In [3]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
import remote_sensing_core as rc
import remote_sensing_plot_core as rcp

start_time = time.time()

# crop_type vs crop_group

In [4]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/01_Data_part_not_filtered/"

In [7]:
shapeFile_2016 = pd.read_csv(data_dir + f_names[0], low_memory = False)
shapeFile_2017 = pd.read_csv(data_dir + f_names[1], low_memory = False)
shapeFile_2018 = pd.read_csv(data_dir + f_names[1], low_memory = False)

In [10]:
shapeFile_2016.head(5)

,ID,CropGrp,CropTyp,Acres,Irrigtn,IntlSrD,LstSrvD,DataSrc,Notes,TRS,county,RtCrpTy,ExctAcr,Organic,Shp_Lng,Shap_Ar
0,1_WSDA_SF_2016,Orchard,Apple,37,Sprinkler,2003/07/23 00:00:00,2016/04/20 00:00:00,WSDA,NaN,T09R27E26,Benton,Asparagus,37.268741,NaN,2571.290736,150821.245614
1,2_WSDA_SF_2016,Vegetable,Sugar Beet,124,Center Pivot,2003/08/19 00:00:00,2016/09/02 00:00:00,WSDA,B-15,T07R26E30,Benton,Bluegrass Seed,123.542346,NaN,2506.790331,499958.134510
2,3_WSDA_SF_2016,Orchard,Apple,1,Sprinkler,2003/06/25 00:00:00,2016/04/20 00:00:00,WSDA,NaN,T09R25E32,Benton,NaN,1.231077,NaN,304.242010,4981.991440
3,4_WSDA_SF_2016,Cereal Grain,Wheat,27,None,2003/10/08 00:00:00,2016/12/31 00:00:00,NASS,NaN,T06R24E29,Benton,Wheat Fallow,26.677254,NaN,1571.027212,107959.014784
4,5_WSDA_SF_2016,Other,Fallow,21,Wheel Line,2003/07/24 00:00:00,2016/07/12 00:00:00,WSDA,NaN,T08R28E17,Benton,Timothy,20.681856,NaN,1288.345452,83696.502165


# Find Acreage per crop group and croup type in Douglas

In [77]:
data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/" + \
           "01_NDVI_TS/70_Cloud/00_Eastern_WA_withYear/2Years/Douglas_MikeBrady/"

param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"

given_county = "Douglas"
SG_win_size = 7
SG_poly_Order = 3

years = [2016, 2017, 2018]

file_names = ["Douglas_2016_regular_EVI_SG_win7_Order3.csv",
              "Douglas_2017_regular_EVI_SG_win7_Order3.csv",
              "Douglas_2018_regular_EVI_SG_win7_Order3.csv"]

In [78]:
double_crop_potential = pd.read_csv(param_dir + "double_crop_potential_plants.csv")

In [79]:
irrigated_only = True
NASS_out = False
only_annuals = True
small_fields_out = False
Last_survey_year = False

In [80]:
remove_columns = ["RtCrpTy", "SOS", "EOS", "human_system_start_time", "Shp_Lng", "Shap_Ar", 
                  "TRS", "doy", "IntlSrD"]

In [90]:
for file in file_names:
    print (file)
    original_data_tble = pd.read_csv(data_dir + file, low_memory = False)
    data_tble = original_data_tble.copy()

    if irrigated_only == True:
        data_tble = rc.filter_out_nonIrrigated(data_tble)
        irrigated_name = "onlyIrrigated"
    else:
        irrigated_name = "irrigatedAndNonIrr"


    if NASS_out == True:
        data_tble = rc.filter_out_NASS(data_tble)
        NASS_out_name = "NASSOut"
    else:
        NASS_out_name = "allSurvySources"


    if only_annuals == True:
        data_tble = data_tble[data_tble.CropTyp.isin(double_crop_potential['Crop_Type'])]
        only_annuals_name = "onlyAnnuals"
    else:
        only_annuals_name = "AnnualsAndPerenials"


    if small_fields_out == True:
        data_tble = data_tble[data_tble.Acres > 3]
        small_fields_name = "bigFields"
    else:
        small_fields_name = "bigAndSmallFields"

    proper_year = file.split("_")[1]
    proper_year = proper_year.split(".")[0]

    if Last_survey_year == True:
        data_tble = data_tble[data_tble['LstSrvD'].str.contains(proper_year)]
        Last_survey_year_name = "correctLstSrvD"
    else:
        Last_survey_year_name = "wrongLstSrvD"

    CropGrp_acreage = data_tble.groupby(["county", "CropGrp", "season_count"]).Acres.sum().reset_index()
    CropType_acreage = data_tble.groupby(["county", "CropTyp", "season_count"]).Acres.sum().reset_index()

    # Saving path
    out_dir = data_dir + "/acreage_tables/"
    out_dir = data_dir
    os.makedirs(out_dir, exist_ok=True)

    CropGrp_acreage_name = out_dir + proper_year + "_" + \
                           "Douglas_CropGrp_doubleAcr_" + \
                            irrigated_name + "_" + \
                            NASS_out_name + "_" + \
                            only_annuals_name + "_" + \
                            small_fields_name + "_" + \
                            Last_survey_year_name + "_" + \
                            ".csv"

    CropType_acreage_name = out_dir + proper_year + "_" + \
                           "Douglas_CropType_doubleAcr_" + \
                            irrigated_name + "_" + \
                            NASS_out_name + "_" + \
                            only_annuals_name + "_" + \
                            small_fields_name + "_" + \
                            Last_survey_year_name + \
                            ".csv"

    CropGrp_acreage.to_csv(CropGrp_acreage_name, index = False)
    CropType_acreage.to_csv(CropType_acreage_name, index = False)

Douglas_2016_regular_EVI_SG_win7_Order3.csv
Douglas_2017_regular_EVI_SG_win7_Order3.csv
Douglas_2018_regular_EVI_SG_win7_Order3.csv


In [89]:
data_tble.head(2)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,...,image_year,SF_year,doy,EVI,human_system_start_time,Date,EVI_ratio,SOS,EOS,season_count
8,82779_WSDA_SF_2018,6,Douglas,Cereal Grain,wheat,nass,5.602117,2003/04/10 00:00:00,center pivot,2015/12/31 00:00:00,...,2018,2018,1,0.154159,2018-01-01 00:00:00,2018-01-01 00:00:00,666.000000,666.0,666.0,1
13,82776_WSDA_SF_2018,81,Douglas,Vegetable,potato,nass,80.516842,2003/01/03 00:00:00,center pivot,2017/12/31 00:00:00,...,2018,2018,1,0.072792,2018-01-01 00:00:00,2018-01-01 00:00:00,0.007011,0.0,0.0,1


In [88]:
CropGrp_acreage

,county,CropGrp,season_count,Acres
0,Douglas,Cereal Grain,1,777
1,Douglas,Cereal Grain,2,140
2,Douglas,Green Manure,1,10
3,Douglas,Hay/Silage,1,23
4,Douglas,Oilseed,1,194
5,Douglas,Seed,1,55
6,Douglas,Vegetable,1,155
